In [62]:
import cv2
import numpy as np

def readImagesAndTimes():
    times = np.array([ 1/30.0, 0.25, 2.5, 15.0 ], dtype=np.float32)
    images = []
    filenames = ["img_0.033.jpg", "img_0.25.jpg", "img_2.5.jpg", "img_15.jpg"]
    for filename in filenames:
        im = cv2.imread(filename)
        images.append(im)
    return images, times

In [63]:
### HDR 


images, times = readImagesAndTimes()

alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)

mergeDebevec = cv2.createMergeDebevec()
ndrDebevec = mergeDebevec.process(images, times, responseDebevec)

ndrDebevec = cv2.resize(ndrDebevec, (1200, 900))
cv2.imwrite('TEST.hdr', ndrDebevec)
cv2.imshow('test', ndrDebevec)
cv2.waitKey(0)

-1

In [66]:
## Тонируем с тспользовагием Drago

tonemapDrago = cv2.createTonemapDrago(1.0, 0.7)
ldrDrago = tonemapDrago.process(ndrDebevec)
ldrDrago = 3 * ldrDrago

cv2.imwrite('TEST_DRAGO.jpg', ldrDrago * 255)
hdr1 = cv2.imread('TEST_DRAGO.jpg')
cv2.imshow('drago', hdr1)
cv2.waitKey(0)

-1

In [67]:
## Тонируем методом Reinhard

tonemapReinhard = cv2.createTonemapReinhard(1.5, 0,0,0)
ldrReinhard = tonemapReinhard.process(ndrDebevec)
cv2.imwrite('TEST_Reinhard.jpg', ldrReinhard * 255)


hdr2 = cv2.imread('TEST_Reinhard.jpg')
cv2.imshow('drago', hdr2)
cv2.waitKey(0)

-1

In [70]:

tonemMantiuk = cv2.createTonemapMantiuk(2.2, 0.85, 1.2)
ldrMantiuk = tonemMantiuk.process(ndrDebevec)
ldrMantiuk = 3 * ldrMantiuk
cv2.imwrite('TEST_Mantiuk.jpg', ldrMantiuk * 255)

hdr3 = cv2.imread('TEST_Mantiuk.jpg')
cv2.imshow('Mauntiuk', hdr3)
cv2.waitKey(0)

-1

In [77]:
import glob

for name in glob.glob('D:/Neural_networks/Lesson_57/**.jpg'):
    img = cv2.imread(name)
    cv2.imshow('res', img)
    if cv2.waitKey(0) & ord('q') == 0xff:
        break
    

In [4]:
import cv2
import numpy as np

img = cv2.imread('D:/Neural_networks/Lesson_57/img/wolf1.jpg')

b, g, r = cv2.split(img)

b_co = cv2.equalizeHist(b)
g_co = cv2.equalizeHist(g)
r_co = cv2.equalizeHist(r)

img_merge = cv2.merge((b_co, g_co, r_co))
cv2.imshow('merge', np.hstack([img_merge, img]))
cv2.waitKey(0)


-1

In [4]:
import glob
import cv2
import numpy as np

original = cv2.imread("D:/Neural_networks/Lesson_57/img/wolf1.jpg")

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype('uint8')
    return cv2.LUT(image, table)

for gamma in np.arange(0.0, 3.5, 0.5):
    if gamma == 1:
        continue
    gamma = gamma if gamma > 0 else 0.1
    adjust = adjust_gamma(original, gamma=gamma)
    cv2.putText(adjust, f"g={gamma}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 3)
    cv2.imshow('change_gamma', np.hstack([adjust, original]))
    cv2.waitKey(0)
    

In [2]:
import cv2
import numpy as np


def readImagesAndTimes():
    times = np.array([1/30.0, 0.25, 2.5, 15.0])
    filenames = ["img_0.033.jpg", "img_0.25.jpg", "img_2.5.jpg", "img_15.jpg"]
    images = []
    for filename in filenames:
        im = cv2.imread(filename)
        images.append(im)
    return images, times
readImagesAndTimes()

([array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         ...,
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [1, 1, 1],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [1, 1, 1],
          [1, 1, 1],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8),
  array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
  

In [ ]:
print("Читання зображень ... ")
images, times = readImagesAndTimes()

# Вирівнюємо вхідні зображення
print("Вирівнюємо зображення ... ")
alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

# Отримуємо функцію відповіді камери (CRF)
print("Обчислюємо функцію відповіді камери (CRF) ... ")
calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)

# Об'єднуємо зображення в лінійне зображення HDR
print("Об'єднуємо зображення в лінійне зображення HDR ... ")
mergeDebevec = cv2.createMergeDebevec()
hdrDebevec = mergeDebevec.process(images, times, responseDebevec)
# Save HDR image.
cv2.imwrite("hdrDebevec.hdr", hdrDebevec)
print("Зберігаємо у hdrDebevec.hdr ")